In [3]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm


In [4]:
train_path_checked = 'febhack2020/datasets/training/checked'
dirs_checked = [dI for dI in os.listdir(train_path_checked) if os.path.isdir(os.path.join(train_path_checked,dI)) and dI != '.ipynb_checkpoints']

train_path_unchecked = 'febhack2020/datasets/training/unchecked'
dirs_unchecked = [dI for dI in os.listdir(train_path_unchecked) if os.path.isdir(os.path.join(train_path_unchecked,dI)) and dI != '.ipynb_checkpoints']

valid_path = 'febhack2020/datasets/validation/'
dirs_valid = [dI for dI in os.listdir(valid_path) if os.path.isdir(os.path.join(valid_path,dI)) and dI != '.ipynb_checkpoints']



In [6]:
def time_persistence(data1a, n, param):

    ts = np.array([data1a['ndvi_avg']]).reshape(-1,1)
    cldfree = np.array([(data1a['SCL=4']+data1a['SCL=5'])/data1a['total']]) #percentage of cloud free pixels
    
    data1a = pd.to_datetime(data1a["t"])
    days=[0]
    for ind in range(1, len(data), 1):
        days +=  [days[ind-1] +(data.loc[ind,"t"] - data.loc[ind-1,"t"]).days]
    
    
    while True:
        
        if n > 0 and n < len(data1a)-1:        
            thresh1=1
            while True:
                tr_fl = cldfree > thresh1
                added = [i for i, x in enumerate(tr_fl[0]) if x] # indices of cloud free days
                try:
                    n_per1 = np.max([i for i in added if i < n])
                    break
                except:
                    thresh1*=0.995
                if thresh1 <= param:
                    n_per1 = 0
                    break
            
            thresh2=1
            while True:
                tr_fl = cldfree > thresh2
                added = [i for i, x in enumerate(tr_fl[0]) if x] # indices of cloud free days
                try:
                    n_per2 = np.min([i for i in added if i > n])
                    break
                except:
                    thresh2*=0.995
                if thresh2 <= param:
                    n_per2 = 0
                    break
        
            if thresh1 <= param and thresh2 <= param:
                cldfree[0,n] = -1
                ind = np.argmax(cldfree)
                return ts[ind]
                                                   
   
            diff1 = (days[n] - days[n_per1]) if thresh1 > param else 100000
            diff2 = (days[n_per2] - days[n]) if thresh2 > param else 100000

            ts[n] = ((thresh1/diff1)*ts[n_per1] + (thresh2/diff2)*ts[n_per2])/(thresh1/diff1+thresh2/diff2)
            return ts[n]

        elif n==0:
            
            thresh=1
            while True:
                tr_fl = cldfree > thresh
                added = [i for i, x in enumerate(tr_fl[0]) if x] # indices of cloud free days
                try:
                    n_per = np.min([i for i in added if i > n])
                    return ts[n_per]
                    
                except:
                    thresh*=0.995          
        else:
            thresh=1
            while True:
                tr_fl = cldfree > thresh
                added = [i for i, x in enumerate(tr_fl[0]) if x] # indices of cloud free days
                try:
                    n_per = np.max([i for i in added if i < n])
                    return ts[n_per]
                    
                except:
                    thresh*=0.995


In [ ]:
values = []
for i in tqdm(range(70,100,1)):
    param = i/100
    loss=0
    for dir_ in dirs_valid:

        data_path = os.path.join(valid_path, dir_, 'data1a.csv')
        target_path = os.path.join(valid_path, dir_, 'answer1a.txt')
        data = pd.read_csv(data_path)
        date = pd.to_datetime(data_path.split("_")[0].split("/")[-1])
        data["t"] = pd.to_datetime(data["t"]).dt.date
        n = data[data["t"] == date].index[0]


        y_pred = time_persistence(data, n, param)
        y_true = float(open(target_path, "r").read())
        loss += (y_pred-y_true)**2

    loss = np.sqrt(loss/len(dirs_valid))
    values += [[i, loss]]
    print(i, loss)

In [7]:
loss=0
for dir_ in tqdm(dirs_valid):

    data_path = os.path.join(valid_path, dir_, 'data1a.csv')
    target_path = os.path.join(valid_path, dir_, 'answer1a.txt')
    data = pd.read_csv(data_path)
    date = pd.to_datetime(data_path.split("_")[0].split("/")[-1])
    data["t"] = pd.to_datetime(data["t"]).dt.date
    n = data[data["t"] == date].index[0]

    y_pred = time_persistence(data, n, 0.97)
    y_true = float(open(target_path, "r").read())
    loss += (y_pred-y_true)**2

loss = np.sqrt(loss/len(dirs_valid))

100%|██████████| 98/98 [00:01<00:00, 92.14it/s]


In [8]:
loss

array([0.01962097])